![](../images/itam_logo.png)

M. Sc. Liliana Millán Núñez liliana.millan@itam.mx

Febrero 2021


## AWS with Python

In [8]:
import boto3 

In [9]:
# saber qué versión de Boto estamos ocupando
print(boto3.__version__)

1.11.15


Si tienes tu archivo `~/.aws/credentials` puedes ocupar dentro del objeto `Session`, qué perfil vas a ocupar para conectarte a AWS, también es posible indicar la región y las credenciales asociadas, aunque esto **no** se recomienda, en este caso es mejor poner las llaves en plano en algún archivo `yaml` en tu computadora y leerlos desde ahí para que **no** subas tus llaves al repositorio!.

#### Con profile

In [3]:
ses = boto3.session.Session(profile_name='mge', region_name='us-west-2')

In [4]:
s3 = ses.resource('s3')

In [5]:
# listar los buckets que están en este perfil y región.
for bucket in s3.buckets.all():
    print(bucket.name)

aplicaciones-cd-1
aplicaciones-cd-2
aws-athena-query-results-639547754660-us-east-1
aws-athena-query-results-639547754660-us-west-2
aws-glue-scripts-639547754660-us-west-2
aws-glue-temporary-639547754660-us-west-2
aws-logs-639547754660-us-east-1
aws-logs-639547754660-us-west-2
cse344-test-file
dpa
dpa-encrypted
dpa-encrypted-2
metodos-gran
metodos-gran-escala
test-secure


### Leyendo llaves de yaml

In [2]:
import yaml

In [3]:
# este archivo yaml vive en tu conf/local/credentials.yaml
def read_yaml_file(yaml_file):
    """ load yaml cofigurations """

    config = None
    try: 
        with open (yaml_file, 'r') as f:
            config = yaml.safe_load(f)
    except:
        raise FileNotFoundError('Couldnt load the file')
    
    return config

In [6]:
def get_s3_credentials(credentials_file):
    credentials = read_yaml_file(credentials_file)
    s3_creds = credentials['s3']

    return s3_creds

In [10]:
#s3_creds = get_s3_credentials("../../conf/local/credentials.yaml")
s3_creds = get_s3_credentials("/home/silil/Documents/itam/metodos_gran_escala/credentials.yaml")

session = boto3.Session(
    aws_access_key_id=s3_creds['aws_access_key_id'],
    aws_secret_access_key=s3_creds['aws_secret_access_key']
)
s3 = session.client('s3')

##### Listado de solo una carpeta en específico

In [15]:
bucket = "metodos-gran-escala"
key = "northwind"

In [16]:
objects = s3.list_objects_v2(Bucket=bucket, Prefix=key)['Contents']

In [17]:
objects

[{'Key': 'northwind/customers.csv',
  'LastModified': datetime.datetime(2019, 2, 27, 1, 57, 6, tzinfo=tzutc()),
  'ETag': '"5f42cfb8c6ce316281596336f87f37bb"',
  'Size': 11558,
  'StorageClass': 'STANDARD'},
 {'Key': 'northwind/employees.csv',
  'LastModified': datetime.datetime(2019, 2, 27, 1, 57, 6, tzinfo=tzutc()),
  'ETag': '"ce832fe36e34dd7942c322322500fef9"',
  'Size': 4059,
  'StorageClass': 'STANDARD'},
 {'Key': 'northwind/order_details.csv',
  'LastModified': datetime.datetime(2019, 3, 6, 1, 57, 10, tzinfo=tzutc()),
  'ETag': '"f493c58c1cb13b6e9888dd091c6d3f10"',
  'Size': 40622,
  'StorageClass': 'STANDARD'},
 {'Key': 'northwind/orders.csv',
  'LastModified': datetime.datetime(2019, 3, 6, 1, 56, 52, tzinfo=tzutc()),
  'ETag': '"cfa9eaf9c9e12e08e617747cf43a3921"',
  'Size': 98517,
  'StorageClass': 'STANDARD'},
 {'Key': 'northwind/products.csv',
  'LastModified': datetime.datetime(2019, 2, 27, 1, 57, 6, tzinfo=tzutc()),
  'ETag': '"4e9ef122f1fe7673e1b0d3643e995d17"',
  'Size':

### Creación de un bucket

+ Por default los *buckets* son creados en la región *US East (N. Virginia)* `us-east-1`, necesitarás definir en qué región quieres crear el *bucket*. 
+ Los nombres de buckets **no** pueden llevar guiónes bajos, te marcará que el nombre es inválido, en lugar de los guiones bajos puedes ocupar guiónes altos.

In [45]:
#crear un bucket
bucket_name = 'test-aws-boto'

s3.create_bucket(Bucket=bucket_name, 
                 CreateBucketConfiguration={'LocationConstraint': 'us-west-2'},
                ACL='private')

s3.Bucket(name='test-aws-boto')

### Guardar archivos en el bucket

In [55]:
# subiremos un archivo csv de ejemplo: pollution con ~43k registros 
#con observaciones de contaminación del aire en una ciudad de China. 
file_to_upload = '/home/silil/Documents/pollution.csv'
file_name = file_to_upload.split(sep='/')[-1]

# accedemos a client desde el resource
s3.meta.client.upload_file(file_to_upload, bucket_name, file_name)

### Acceder al bucket

In [56]:
# acceder al bucket
my_bucket = s3.Bucket(bucket_name)

for element in my_bucket.objects.all():
    print(element)

s3.ObjectSummary(bucket_name='test-aws-boto', key='pollution.csv')


### Bajar archivos del bucket

In [60]:
path_to_download = '/home/silil/Documents/itam/metodos_gran_escala/data-product-architecture/' + file_name

s3.meta.client.download_file(bucket_name, file_name, path_to_download)

### Borrar archivos del bucket

In [61]:
s3.Object(bucket_name, file_name).delete()

{'ResponseMetadata': {'RequestId': 'F6D1AA9745ADFEBE',
  'HostId': '6SAzJwWvNHxweiJBD8hBuFyfl9/HdyS+8R4uHkOZIjlMaDPjqosrqbQqinpoqywoYH3iVdNNkJE=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': '6SAzJwWvNHxweiJBD8hBuFyfl9/HdyS+8R4uHkOZIjlMaDPjqosrqbQqinpoqywoYH3iVdNNkJE=',
   'x-amz-request-id': 'F6D1AA9745ADFEBE',
   'date': 'Fri, 14 Feb 2020 20:56:53 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

### Borrar el bucket

In [63]:
s3.Bucket(bucket_name).delete()

{'ResponseMetadata': {'RequestId': 'AD0D410932E0AF53',
  'HostId': 'uyoHxHQlhtK4SnDgxU0DPNGfrdg+8DOmLR4dda7bY9sLApMf/JwzXkHQEuaO5CgjxVK9vA5gHXY=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'uyoHxHQlhtK4SnDgxU0DPNGfrdg+8DOmLR4dda7bY9sLApMf/JwzXkHQEuaO5CgjxVK9vA5gHXY=',
   'x-amz-request-id': 'AD0D410932E0AF53',
   'date': 'Fri, 14 Feb 2020 20:57:52 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

### Referecias 

+ [S3 boto3 API Reference](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html)